In [11]:
## This is a demo for atlas
## Author : Avadesh Meduri
## Date : 06/04/2022
%load_ext autoreload
%autoreload 2

import time
import numpy as np
from robot_properties_anymal.config import AnymalConfig
from mpc.anymal_cyclic_gen import AnymalMpcGaitGen

from matplotlib import pyplot as plt

import pinocchio as pin

import numpy as np
from motions.weight_abstract import BiconvexMotionParams

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
robot = AnymalConfig.buildRobotWrapper()
rmodel = robot.model
rdata = robot.data

pin_robot = AnymalConfig.buildRobotWrapper()

viz = pin.visualize.MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)
viz.initViewer(open=False)
viz.loadViewerModel()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7006/static/


In [13]:
jump = BiconvexMotionParams("anymal", "Jump")

# Cnt
jump.gait_period = 0.5
jump.stance_percent = [0.4, 0.4, 0.4, 0.4]
jump.gait_dt = 0.05
jump.phase_offset = [0.3, 0.3, 0.3, 0.3]

# IK
jump.state_wt = np.array([0., 0, 10] + [1000, 1000, 1000] + [1e3] * (pin_robot.model.nv - 6) \
                        + [0.00] * 3 + [100, 100, 100] + [5e0] *(pin_robot.model.nv - 6))

jump.ctrl_wt = [0, 0, 1000] + [5e2, 5e2, 5e2] + [10.0] *(pin_robot.model.nv - 6)

jump.swing_wt = [1e5, 1e4]
jump.cent_wt = [3*[0*5e+1], 6*[5e+1]]
jump.step_ht = 0.05
jump.nom_ht = 0.34
jump.reg_wt = [5e-2, 1e-5]

# Dyn 
jump.W_X =        np.array([1e-5, 1e-5, 1e+5, 1e+1, 1e+1, 2e+2, 1e+4, 1e+4, 1e4])
jump.W_X_ter = 10*np.array([1e+5, 1e-5, 1e+5, 1e+1, 1e+1, 2e+2, 1e+5, 1e+5, 1e+5])
jump.W_F = np.array(4*[1e+2, 1e+2, 1e+2])
jump.rho = 5e+4
jump.ori_correction = [0.3, 0.5, 0.3]
jump.gait_horizon = 10.0
jump.kp = 150.0
jump.kd = 10.0


In [14]:
## Motion
gait_params = jump

## robot config and init
urdf_path = AnymalConfig.urdf_path
gait_time = gait_params.gait_period
dt = 5e-2

n_eff = 4
q0 = np.array(AnymalConfig.initial_configuration)
# q0[7:13] = 2 * [0.0, -0.8, 1.6]

v0 = pin.utils.zero(pin_robot.model.nv)
x0 = np.concatenate([q0, pin.utils.zero(pin_robot.model.nv)])

w_des = 0.0
v_des = np.array([0.2,0.0, 0])

update_time = 0.02 # sec (time of lag)

n = 1

sim_t = 0.0
step_t = 0
sim_dt = .001
plan_freq = 5.0

gg = AnymalMpcGaitGen(pin_robot, urdf_path, x0, plan_freq, q0, None)
gg.update_gait_params(gait_params, sim_t)

Initialized Kino-Dyn planner


In [16]:
viz.viewer.jupyter_cell()

In [20]:
%matplotlib notebook 

q = q0
v = v0
step_t = 0
n = 0

xs_plan, us_plan, f_plan = gg.optimize(q, v, np.round(sim_t,3), v_des, w_des)
# gg.plot(q, v)
for ind in range(int(xs_plan.shape[0])):
    viz.display(xs_plan[ind][:robot.model.nq])

Cost Time :Maximum iterations reached 
Final norm: 0.0396784
 0.03402590751647949
Solve Time :  0.20749545097351074
